In [189]:
import pandas as pd
import numpy as np

df = pd.read_csv("/home/bosser/manfred2025-02-14 12:00:29.csv")

In [191]:
df

,url,fecha_extraccion,titulo,oferta_activada,detalles_1_SALARIO,detalles_1_PRESENCIAL,detalles_1_TELETRABAJO,detalles_2_DÍA LABORABLE,detalles_2_VACACIONES,detalles_2_JORNADA LABORAL,...,tecnologias_Tecnologías_Lua,tecnologias_Tecnologías_Zend,tecnologias_Tecnologías_CircleCI,tecnologias_Tecnologías_Serverless,tecnologias_Tecnologías_Zapier,tecnologias_Tecnologías_Apache Tomcat,tecnologias_Tecnologías_git-bash,tecnologias_Tecnologías_Microsoft WPF,tecnologias_Tecnologías_Adobe Launch,tecnologias_Tecnologías_Adobe Target
0,https://www.getmanfred.com/ofertas-empleo/6744...,2025-02-14 11:10:38,Python 3D Developer,Activa,HASTA €40K,MADRID,30%,JORNADA COMPLETA,25 DÍAS,FLEXIBLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.getmanfred.com/ofertas-empleo/6677...,2025-02-14 11:10:49,Product Manager,Activa,€60-70K,BARCELONA,60%,JORNADA COMPLETA,23 DÍAS,FLEXIBLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.getmanfred.com/ofertas-empleo/6675...,2025-02-14 11:11:21,Digital Marketing Specialist,Activa,HASTA €24K,MADRID,60%,JORNADA COMPLETA,23 DÍAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.getmanfred.com/ofertas-empleo/4193...,2025-02-14 11:11:33,Tech Lead,Activa,€55-65K,MADRID,NaN,JORNADA COMPLETA,23 DÍAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.getmanfred.com/ofertas-empleo/6574...,2025-02-14 11:11:45,.NET Developer,Activa,€40-48K,MADRID OR SEVILLA OR PALMA,60%,JORNADA COMPLETA,23 DÍAS,FLEXIBLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,https://www.getmanfred.com/ofertas-empleo/27/p...,2025-02-14 12:00:22,Frontend Developer Junior,Oferta cerrada el 24/09/2021,€24-26K,NaN,NaN,NaN,22 DÍAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,https://www.getmanfred.com/ofertas-empleo/23/f...,2025-02-14 12:00:23,Senior Backend Developer,Oferta cerrada el 01/09/2021,€50-60K,MADRID,NaN,NaN,22 DÍAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1223,https://www.getmanfred.com/ofertas-empleo/21/m...,2025-02-14 12:00:25,Senior Java Developer,Oferta cerrada el 08/07/2022,€45-55K,MADRID,NaN,NaN,30 DÍAS,FLEXIBLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1224,https://www.getmanfred.com/ofertas-empleo/18/m...,2025-02-14 12:00:27,Solution Architect,Oferta cerrada el 08/08/2022,HASTA €80K,NaN,NaN,NaN,30 DÍAS,FLEXIBLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
#Crea columna id y lo lleva a la primera posición

df["id"] = ["id_man_" + str(i) for i in range(1, len(df) + 1)]

columna_extraida_ = df.pop("id")

df.insert(0, "id", columna_extraida_)



#Nueva columna con el nombre de la empresa a partir de la url y la mueve a la segunda posición

df["empresa"] = df["url"].apply(lambda x: x.split("/")[-1].split("-")[0])

columna_extraida = df.pop("empresa")

df.insert(1, "empresa", columna_extraida)



#Elimina "tecnologias_Tecnologías_" de todos los nomnbres de columna

df.columns = df.columns.str.replace("tecnologias_Tecnologías_", "", regex=False)



#Renombra columnas que empiezan por detalles_1_, detalles_2_

df = df.rename(columns={"detalles_1_SALARIO" : "Salario", "detalles_1_PRESENCIAL" : "Presencial", "detalles_1_TELETRABAJO" : "Teletrabajo", "detalles_1_REMOTO" : "Remoto", "detalles_2_DÍA LABORABLE" : "dia_laborable", "detalles_2_VACACIONES" : "vacaciones", "detalles_2_JORNADA LABORAL" : "jornada_laboral", "detalles_2_TURNO CONTINUO" : "turno_continuo", "detalles_1_VARIABLE" : "variable"})



#Quitar " DÍAS" en columna vacaciones
###Pendiente revisar ILIMITADO

df["vacaciones"] = df["vacaciones"].str.replace(" DÍAS", "")
df["vacaciones"].value_counts()
# Reemplazar "ILIMITADO" con NaN
df.replace("ILIMITADO", np.nan, inplace=True)



#Columna habilidades_Otras habilidades. 

# Convierte los strings a listas.
import ast
df["habilidades_Otras habilidades"] = df["habilidades_Otras habilidades"].apply(ast.literal_eval)

#Crea una lista con las habilidades únicas
todas_habilidades = sum(df["habilidades_Otras habilidades"], [])
habilidades_unicas = list(set(todas_habilidades))

# Crea una columna para cada habilidad única y usa la columna original para poblar las nuevas conlumnas con 1 o 0
for habilidad in habilidades_unicas:
    df[habilidad] = df["habilidades_Otras habilidades"].apply(lambda x: 1 if habilidad in x else 0)

#Elimina la columna habilidades_Otras habilidades. 

df = df.drop("habilidades_Otras habilidades", axis = 1)

In [195]:
#Dividimos las columnas en 3 categorías y generamos un df para cada uno de ellos.

programas = ["id", "Python", "C++", "Cinema 4D", "CRM", "Canva", "Trello", "Google Analytics", "Adobe Photoshop", 
    "Adobe Illustrator", "WordPress", "Java", "SQL", "SOAP", "OpenApi", "JavaScript", "Cobol", 
    "C#", ".NET", "JSON", "Angular2+", "Microsoft Excel", "Hubspot", "AWS", "Kafka", "Argo CD", 
    "Flux", "React", "Kotlin", "Swift", "PostgreSQL", "Spring", "Azure", "Kubernetes", "Terraform", 
    "Docker", "Linux", "Git", "Golang", "Rust", "GitLab", "Django", "Celery", "Fastapi", "MySQL", 
    "NodeJS", "TypeScript", "MVC", "dbt", "Google Cloud", "Airflow", "z/OS", "DB2", "PowerShell", 
    "Bash", "Magento", "Prestashop", "BigCommerce", "Salesforce", "Shopify", "WooCommerce", 
    "RabbitMQ", "NextJS", "Tailwind", "Jenkins", "Databricks", "Prometheus", "Google Sheets", "nestJS", 
    "Vue", "MQTT", "Express", "Koa", "Spring Boot", "Go", "Bootstrap", "Pytest", "Github", "PHP", 
    "Symfony", "Figma", "HTML", "CSS", "Jest", "Hibernate", "API", "WebSockets", "Svelte", "BigQuery", 
    "Redux", "Ansible", "SQLServer", "Visual Studio", "REST", "ETL", "S3", "Redshift", "Snowflake", 
    "Auth0", "Tableau", "IIS", "Pulumi", "Azure DevOps", "Laravel", "Yii", "AWS ECS", "Nest", "PySpark", 
    "AWS Lambda", "Selenium", "Appium", "Cypress", "RxJS", "Clojure", "Grafana", "ELK", "Spark", "Jmeter", 
    "Gatling", "Apple iOS", "SwiftUI", "XCTest", "UIKit", "Atlassian", "Groovy", "ServiceNow", "Ethereum", 
    "Web3", ".Net Core", "Swagger", "Helm", "Datadog", "Kong", "Quarkus", "Vitest", "TestingLibrary", 
    "Metabase", "Webpack", "Elasticsearch", "Dart", "Flutter", "Blender", "Ruby on Rails", "Citrix", 
    "Windows", "VMWare", "Hyper-v", "Xen", "R", "Three.js", "MongoDB", "React Native", "Jira", "NoSQL", 
    "Scrapy", "Apache Beam", "Nagios", "Zabbix", "DNS", "HTTP4s", "Razor", "Shiny", "Xray", "AWS Cloudwatch", 
    "Google Ads", "Pipedrive", "Notion", "Monday", "DialogFlow", "GraphQL", "jQuery", "AngularJS", "Ionic", 
    "Viper", "Android", "Slack", "Mocha", "junit4", "Amplitude", "Mixpanel", "DynamoDB", "SaaS", "Redis", 
    "KVM", "Solr", "Flask", "Sklearn", "Google Looker Studio", "Scala", "Gitflow", "ASP.net", "Pandas", 
    "Numpy", "Google Firebase", "Riverpod", "gRPC", "Protobuf", "PHPunit", "Drupal", "Zoho", "Fortran", 
    "Vite", "TensorFlow", "ActiveMQ", "Cisco", "Microsoft Sharepoint", "Microsoft Dynamics 365", "Cassandra", 
    "Dagger2", "Core Data", "Ruby", "PyTorch", "AWS SageMaker", "Oracle", "Microsoft Power BI", 
    "Google Tag Manager", "Maven", "JUnit", "Blazor", "Microsoft", "javacript", "Sketch", "Adobe Xd", 
    "Axure", "InfluxDB", "Flink", "Postman", "JBoss", "Transact-SQL (T-SQL)", "PL/SQL", "Cordova", 
    "BlueTooth", "Nuxt.js", "Babel", "Microsoft Office 365", "Objective-C", "OpenCV", "Jasmine", 
    "Productboard", "Github Actions", "openlayers", "CARTO", "ArcGIS", "Leaflet", "Geoserver", "Google AdManager", 
    "Google Data Studio", "Google Optimize", "Couchbase", "Elixir", "Adobe Premiere Pro", "Cucumber", 
    "Sagas", "Apollo", "MVVM", "ClickHouse", "Micronaut Framework", "Airtable", "Asana", "Specflow", "Puppeteer", 
    "Miro", "Backbone", "CMS", "KendoUI", "Adobe Suit", "Apache", "Karate", "PACT", "MariaDB", "TestNg", 
    "AWS-User", "Apple macOS", "Microsoft Active Directory", "Storybook", "Keycloak", "slq-server", "GraalVM", 
    "FlexBox", "Unix", "data-lake", "shell-scripting", "postgres", "rabbit", "reactive-programming", "Puppet", 
    "Unity3D", "SAP", "Nginx", "Invision", "Chef", "Hotjar", "LookBack", "Maze", "Webflow", "CMake", "Qt", 
    "Alfresco", "MapBox", "ChartJS", "Google Maps", "OpenGL", "UML", "Genesys", "SIP", "Vuex", "VanillaJS", 
    "Hadoop", "OpenShift", "Hazelcast", "Android Studio", "PowerCenter", "Microsoft SQL Integration Services (SSIS)", 
    "Sass", "Ember", "Entity", "CodeIgniter", "OpenStack", "NgRx", "WPF", "FlaUI", "Gherkin", "Playwright", 
    "SonarQube", "Nexus", "Fortinet", "Vagrant", "F5", "Aruba", "Infoblox", "Palo Alto", "Zscaler", "MobileIron", 
    "PRTG", "VMware NSX", "WebComponents", "New Relic", "Confluence", "Plesk", "AJAX", "SCADA", "Kibana", 
    "Microstrategy", "BI", "Adobe Analytics", "Adobe Campaign", "Selligent", "screen-scraping", "TestFlight", 
    "Odoo", "WebApi", "RollupJS", "YAML", "Lucene", "RedHat", "Segment", "Facebook Ads", "Linkedin Ads", 
    "Bitbucket", "Lua", "Zend", "CircleCI", "Serverless", "Zapier", "Apache Tomcat", "git-bash"]

habilidades = ["id", "Pensamiento creativo", "Tolerancia a la incertidumbre", "Resistencia a la frustración", 
    "Comunicación verbal", "Capacidad de escucha", "Capacidad de mentorización", "Capacidad de dar feedback", 
    "Adaptabilidad al cambio", "Visión estratégica", "Proactividad", "Capacidad de recibir feedback", 
    "Trabajo en equipo", "Habilidades de negociación", "Liderazgo", "Atención al detalle", 
    "Inteligencia emocional", "Capacidad de abstracción", "Capacidad de presentación", "Comunicación escrita", 
    "Pensamiento analítico", "Comunicación intercultural", "Autonomía en el aprendizaje", "Capacidad de autogestión", 
    "Aprendizaje Continuo", "Gestión de equipo", "Visión crítica"]


general = ["id", "empresa", "url", "fecha_extraccion", "titulo", "oferta_activada", "Salario", "Presencial", 
    "Teletrabajo", "dia_laborable", "vacaciones", "jornada_laboral", "variable", "turno_continuo", "Remoto"]




programas_df = df[programas]

habilidades_df = df[habilidades]

general_df = df[general]


In [197]:
#Poblamos programas_df con boolenos en función de lo que la oferta solicita

programas_df.loc[:,programas_df.columns != "id"] = programas_df.loc[:,programas_df.columns != "id"].fillna(0)
programas_df.loc[:,programas_df.columns != "id"] = programas_df.loc[:,programas_df.columns != "id"].map(lambda x: 1 if x != 0 else 0)

In [198]:
habilidades_df

,id,Pensamiento creativo,Tolerancia a la incertidumbre,Resistencia a la frustración,Comunicación verbal,Capacidad de escucha,Capacidad de mentorización,Capacidad de dar feedback,Adaptabilidad al cambio,Visión estratégica,...,Capacidad de abstracción,Capacidad de presentación,Comunicación escrita,Pensamiento analítico,Comunicación intercultural,Autonomía en el aprendizaje,Capacidad de autogestión,Aprendizaje Continuo,Gestión de equipo,Visión crítica
0,id_man_1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,id_man_2,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,id_man_3,0,0,0,1,0,0,0,1,1,...,0,0,0,0,0,1,1,0,0,0
3,id_man_4,0,0,0,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,1,0
4,id_man_5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,id_man_1222,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,id_man_1223,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223,id_man_1224,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1224,id_man_1225,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
programas_df

,id,Python,C++,Cinema 4D,CRM,Canva,Trello,Google Analytics,Adobe Photoshop,Adobe Illustrator,...,Facebook Ads,Linkedin Ads,Bitbucket,Lua,Zend,CircleCI,Serverless,Zapier,Apache Tomcat,git-bash
0,id_man_1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_man_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_man_3,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,id_man_4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_man_5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,id_man_1222,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,id_man_1223,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223,id_man_1224,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1224,id_man_1225,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [204]:
#Sacamos los emojis de la columna titulo

import emoji

general_df["titulo"] = general_df["titulo"].apply(lambda x: emoji.replace_emoji(x, replace="")) 

/tmp/ipykernel_14904/3761228294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["titulo"] = general_df["titulo"].apply(lambda x: emoji.replace_emoji(x, replace=""))


In [205]:
#Modificamos la columna oferta activada a booleanos

general_df["oferta_activada"] = general_df["oferta_activada"].apply(lambda x: 1 if x == "Activa" else 0)

/tmp/ipykernel_14904/2035298039.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["oferta_activada"] = general_df["oferta_activada"].apply(lambda x: 1 if x == "Activa" else 0)


In [208]:
#Limpieza columna Salario

general_df["Salario"] =  general_df["Salario"].str.replace("DESDE €", "").str.replace("HASTA €","").str.replace("€","").str.replace("K","").str.replace("£", "")
general_df["Salario"] =  general_df["Salario"].str.replace("HASTA \u202f", "").str.replace("HASTA US$", "").str.replace("US$", "").str.replace("MXN$", "")

/tmp/ipykernel_14904/1224580849.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["Salario"] =  general_df["Salario"].str.replace("DESDE €", "").str.replace("HASTA €","").str.replace("€","").str.replace("K","").str.replace("£", "")
/tmp/ipykernel_14904/1224580849.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["Salario"] =  general_df["Salario"].str.replace("HASTA \u202f", "").str.replace("HASTA US$", "").str.replace("US$", "").str.replace("MXN$", "")


In [209]:
general_df[["Salario_desde", "Salario_hasta"]] = general_df["Salario"].str.split("-", expand= True)

/tmp/ipykernel_14904/2498840967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df[["Salario_desde", "Salario_hasta"]] = general_df["Salario"].str.split("-", expand= True)
/tmp/ipykernel_14904/2498840967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df[["Salario_desde", "Salario_hasta"]] = general_df["Salario"].str.split("-", expand= True)


In [210]:
general_df["Salario_hasta"] = general_df["Salario_hasta"].fillna(general_df["Salario_desde"])

/tmp/ipykernel_14904/39308143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["Salario_hasta"] = general_df["Salario_hasta"].fillna(general_df["Salario_desde"])


In [217]:
general_df[["Salario_desde", "Salario_hasta"]]

,Salario_desde,Salario_hasta
0,40,40
1,60,70
2,24,24
3,55,65
4,40,48
...,...,...
1221,24,26
1222,50,60
1223,45,55
1224,80,80


In [219]:
general_df[["Salario_desde", "Salario_hasta"]] = general_df[["Salario_desde", "Salario_hasta"]].astype("float")
general_df.drop(columns = ["Salario"], inplace = True)

#Limpieza columna turno_continuo  

general_df["turno_continuo"] = general_df["turno_continuo"].apply(lambda x : "No" if pd.isna(x) else x)

#Limpieza de variable y la lleva hasta la posición 7, al lado de sueldo

general_df["variable"] = general_df["variable"].str.replace("+€","").str.replace("K","").str.replace("+\u202f€","").str.replace(",",".").astype("float")
general_df["variable"] = general_df["variable"].fillna(0)


columna_variable_extraida = general_df.pop("variable")

general_df.insert(7, "variable", columna_variable_extraida)

/tmp/ipykernel_14904/2913254113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df[["Salario_desde", "Salario_hasta"]] = general_df[["Salario_desde", "Salario_hasta"]].astype("float")
/tmp/ipykernel_14904/2913254113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df.drop(columns = ["Salario"], inplace = True)
/tmp/ipykernel_14904/2913254113.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [221]:
#Basandose en las columnas teletrabajo y Remoto actualizamos la columna teletrabajo con el porcentaje de teletrabajo
#Que permite la oferta
#Drop de la columna remoto

general_df["Teletrabajo"] = general_df.apply(lambda x : "100%" if pd.isna(x ["Teletrabajo"]) and x["Remoto"] == "100%" else ("0" if pd.isna(x["Teletrabajo"]) and pd.isna(x["Remoto"]) else x["Teletrabajo"]), axis=1)

general_df["Teletrabajo"] = general_df["Teletrabajo"].str.replace("%","").astype("int")

general_df = general_df.drop("Remoto", axis = 1)

/tmp/ipykernel_14904/3071625530.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["Teletrabajo"] = general_df.apply(lambda x : "100%" if pd.isna(x ["Teletrabajo"]) and x["Remoto"] == "100%" else ("0" if pd.isna(x["Teletrabajo"]) and pd.isna(x["Remoto"]) else x["Teletrabajo"]), axis=1)
/tmp/ipykernel_14904/3071625530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["Teletrabajo"] = general_df["Teletrabajo"].str.replace("%","").astype("int")


In [223]:
#Cambiamos las vacaciones ilimitadas a nulas
general_df["vacaciones"] = general_df["vacaciones"].replace("días ilimitados", np.nan)

In [225]:
#Cambio de jornadas
general_df["Jornada"] = np.where(
    (general_df["jornada_laboral"].isna()) & (general_df["dia_laborable"] == "JORNADA COMPLETA"), "JORNADA COMPLETA",
    np.where(
        (general_df["jornada_laboral"] == "FLEXIBLE") & (general_df["dia_laborable"] == "JORNADA COMPLETA"), "JORNADA COMPLETA FLEXIBLE",
            np.where(
                (general_df["jornada_laboral"].isna()) & (general_df["dia_laborable"].isna()), "JORNADA COMPLETA",
                "JORNADA COMPLETA")))

general_df.drop(columns = ["jornada_laboral", "dia_laborable"], inplace = True)

In [227]:
general_df["turno_continuo"] = np.where(general_df["turno_continuo"] == "No", 0, 1)

In [229]:
general_df

,id,empresa,url,fecha_extraccion,titulo,oferta_activada,Presencial,variable,Teletrabajo,vacaciones,turno_continuo,Salario_desde,Salario_hasta,Jornada
0,id_man_1,geniova,https://www.getmanfred.com/ofertas-empleo/6744...,2025-02-14 11:10:38,Python 3D Developer,1,MADRID,0.0,30,25,0,40.0,40.0,JORNADA COMPLETA FLEXIBLE
1,id_man_2,novicap,https://www.getmanfred.com/ofertas-empleo/6677...,2025-02-14 11:10:49,Product Manager,1,BARCELONA,0.0,60,23,0,60.0,70.0,JORNADA COMPLETA FLEXIBLE
2,id_man_3,psiconnea,https://www.getmanfred.com/ofertas-empleo/6675...,2025-02-14 11:11:21,Digital Marketing Specialist,1,MADRID,2.0,60,23,1,24.0,24.0,JORNADA COMPLETA
3,id_man_4,eri,https://www.getmanfred.com/ofertas-empleo/4193...,2025-02-14 11:11:33,Tech Lead,1,MADRID,0.0,0,23,0,55.0,65.0,JORNADA COMPLETA
4,id_man_5,ust,https://www.getmanfred.com/ofertas-empleo/6574...,2025-02-14 11:11:45,.NET Developer,1,MADRID OR SEVILLA OR PALMA,0.0,60,23,1,40.0,48.0,JORNADA COMPLETA FLEXIBLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,id_man_1222,product,https://www.getmanfred.com/ofertas-empleo/27/p...,2025-02-14 12:00:22,Frontend Developer Junior,0,NaN,0.0,100,22,0,24.0,26.0,JORNADA COMPLETA
1222,id_man_1223,foundernest,https://www.getmanfred.com/ofertas-empleo/23/f...,2025-02-14 12:00:23,Senior Backend Developer,0,MADRID,0.0,100,22,0,50.0,60.0,JORNADA COMPLETA
1223,id_man_1224,mindcurv,https://www.getmanfred.com/ofertas-empleo/21/m...,2025-02-14 12:00:25,Senior Java Developer,0,MADRID,0.0,100,30,0,45.0,55.0,JORNADA COMPLETA
1224,id_man_1225,mindcurv,https://www.getmanfred.com/ofertas-empleo/18/m...,2025-02-14 12:00:27,Solution Architect,0,NaN,0.0,100,30,0,80.0,80.0,JORNADA COMPLETA


In [12]:
#general_df.to_csv("general_df.csv", index=False)
#habilidades_df.to_csv("habilidades_df.csv", index=False)
#programas_df.to_csv("programas_df.csv", index=False)

In [13]:
#####################################################################################

### DUDAS:

En la columna **Vacaciones**, casi todo son valores numéricos, excepto 23 ofertas que ofrecen **días ilimitados**.

Para que la columna pueda ser numérica, se nos ocurre:

- Cambiar los **ILIMITADO** por el máximo de días en otras ofertas.
- Poner otro valor (por ejemplo, `nan`, `-1`, etc.), teniéndolo en cuenta más adelante a la hora de usarlo para gráficas, etc.


In [22]:
#DF solo con ofertas con vacaciones ilimitadas para ver ejemplos.

df_vacaciones_ilimitado = general_df[general_df["vacaciones"] == "ILIMITADO"]

df_filtrado

,id,empresa,url,fecha_extraccion,titulo,oferta_activada,Salario,variable,Presencial,Teletrabajo,dia_laborable,vacaciones,jornada_laboral,turno_continuo
11,id_man_12,foundernest,https://www.getmanfred.com/ofertas-empleo/6196...,2025-02-14 11:13:06,Data Engineer,1,55.0,0.0,NaN,100,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No
73,id_man_74,crossmint,https://www.getmanfred.com/ofertas-empleo/4683...,2025-02-14 11:20:00,Senior Full-stack Engineer,0,95.0,0.0,MADRID,40,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No
80,id_man_81,guruwalk,https://www.getmanfred.com/ofertas-empleo/4410...,2025-02-14 11:20:36,Senior Data Analyst,0,60.0,0.0,VALENCIA,0,JORNADA COMPLETA,ILIMITADO,NaN,No
165,id_man_166,crossmint,https://www.getmanfred.com/ofertas-empleo/3405...,2025-02-14 11:24:38,Senior Frontend Developer,0,80.0,0.0,MADRID,80,NaN,ILIMITADO,FLEXIBLE,No
350,id_man_351,nexthink,https://www.getmanfred.com/ofertas-empleo/1334...,2025-02-14 11:30:57,Head of Engineering for Spain,0,150.0,0.0,MADRID,0,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No
358,id_man_359,nflux,https://www.getmanfred.com/ofertas-empleo/1037...,2025-02-14 11:31:13,Computer Vision Engineer,0,75.0,0.0,NaN,100,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No
359,id_man_360,nflux,https://www.getmanfred.com/ofertas-empleo/1035...,2025-02-14 11:31:15,Senior DevOps Engineer,0,75.0,0.0,NaN,100,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No
372,id_man_373,zeal,https://www.getmanfred.com/ofertas-empleo/998/...,2025-02-14 11:31:42,Product Manager,0,55.0,0.0,NaN,100,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,VIERNES
393,id_man_394,nexthink,https://www.getmanfred.com/ofertas-empleo/950/...,2025-02-14 11:32:25,Senior Director of Engineering,0,130.0,0.0,MADRID,60,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No
430,id_man_431,nflux,https://www.getmanfred.com/ofertas-empleo/579/...,2025-02-14 11:33:43,Python Backend Developer,0,65.0,0.0,NaN,100,JORNADA COMPLETA,ILIMITADO,FLEXIBLE,No


### DUDAS: RESUELTO


En la columna **Presencial**, ¿qué hacemos con las filas en las que hay varias sedes?


In [23]:
df["Presencial"]

0                           MADRID
1                        BARCELONA
2                           MADRID
3                           MADRID
4       MADRID OR SEVILLA OR PALMA
                   ...            
1221                           NaN
1222                        MADRID
1223                        MADRID
1224                           NaN
1225                           NaN
Name: Presencial, Length: 1226, dtype: object

### DUDAS:

**Columnas**: **Día laborable** y **Jornada laboral**.

- Solo hay **jornada completa** o `nan`. ¿Asumimos que todas son jornada completa?
- La mayoría de los que son `nan` en las dos columnas son **100% teletrabajo**.


In [15]:
listadia_laborable = [i for i in general_df["dia_laborable"]]
listadia_laborableunicos = set(listadia_laborable)
listadia_laborableunicos

{'JORNADA COMPLETA', nan}

In [29]:
#Anuncios en los que tanto dia laborable como jornada laboral son nan. Añadimmos %teletrabajo y url para ver relaciones.

for i in range(len(general_df)):
    if pd.isna(general_df[?dia_laborable?][i])  and pd.isna(general_df[?jornada_laboral?][i]):
        print(general_df["Presencial"][i], general_df[?Teletrabajo?][i], general_df[?dia_laborable?][i], general_df[?jornada_laboral?][i], general_df["url"][i])

nan 100 nan nan https://www.getmanfred.com/ofertas-empleo/5991/smowltech-senior-full-stack-phpvue-developer-feb25
BARCELONA 60 nan nan https://www.getmanfred.com/ofertas-empleo/4165/ooptimo-senior-php-developer-oct24
MADRID 40 nan nan https://www.getmanfred.com/ofertas-empleo/4549/todo-en-cloud-product-owner-ago24
nan 100 nan nan https://www.getmanfred.com/ofertas-empleo/3551/bejaofit-senior-flutter-developer-mar24
nan 100 nan nan https://www.getmanfred.com/ofertas-empleo/3502/joinup-senior-python-developer-feb24
nan 100 nan nan https://www.getmanfred.com/ofertas-empleo/2683/237-global-senior-site-reliability-engineer-nov23
nan 100 nan nan https://www.getmanfred.com/ofertas-empleo/1946/world-wide-mobility-ios-mobile-developer-may23
nan 100 nan nan https://www.getmanfred.com/ofertas-empleo/1008/bq-educacion-mid-qa-automation-32h-week-nov22
MADRID O SEVILLA O VALENCIA 100 nan nan https://www.getmanfred.com/ofertas-empleo/958/coverwallet-senior-software-engineer-oct22
nan 100 nan nan http

In [23]:
#  %teletrabajo, dia laborable, jornada laboral y url de todas las ofertas

for i in range(len(general_df)):
    print(general_df["Teletrabajo"][i], general_df["dia_laborable"][i], general_df["jornada_laboral"][i], general_df["url"][i],general_df["id"][i])

30 JORNADA COMPLETA FLEXIBLE https://www.getmanfred.com/ofertas-empleo/6744/geniova-python-3d-developer-feb25 id_man_1
60 JORNADA COMPLETA FLEXIBLE https://www.getmanfred.com/ofertas-empleo/6677/novicap-product-manager-feb25 id_man_2
60 JORNADA COMPLETA nan https://www.getmanfred.com/ofertas-empleo/6675/psiconnea-digital-marketing-specialist-feb25 id_man_3
0 JORNADA COMPLETA nan https://www.getmanfred.com/ofertas-empleo/4193/eri-bancaire-tech-lead-feb25 id_man_4
60 JORNADA COMPLETA FLEXIBLE https://www.getmanfred.com/ofertas-empleo/6574/ust-net-developer-ene25 id_man_5
40 JORNADA COMPLETA nan https://www.getmanfred.com/ofertas-empleo/6572/aplanet-junior-customer-success-ene25 id_man_6
60 JORNADA COMPLETA FLEXIBLE https://www.getmanfred.com/ofertas-empleo/6503/nexthink-java-backend-developer-feb25 id_man_7
100 JORNADA COMPLETA FLEXIBLE https://www.getmanfred.com/ofertas-empleo/6436/mindcurv-mobile-developer-feb25 id_man_8
20 JORNADA COMPLETA FLEXIBLE https://www.getmanfred.com/ofertas-e

In [22]:
#quedan jornada_laboral, dia_laborable, y decidir que hacer con la columna Presencial 